# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.235986313137                   -0.50    8.0         
  2   -7.250366627603       -1.84       -1.39    1.0   8.02ms
  3   -7.251037098114       -3.17       -1.96    1.0   7.67ms
  4   -7.251047392690       -4.99       -1.92    2.0   9.49ms
  5   -7.251327274544       -3.55       -2.58    1.0   8.25ms
  6   -7.251338000087       -4.97       -3.14    1.0   8.05ms
  7   -7.251338721780       -6.14       -3.66    2.0   9.21ms
  8   -7.251338788841       -7.17       -4.08    2.0   9.86ms
  9   -7.251338797829       -8.05       -4.74    2.0   9.27ms
 10   -7.251338798582       -9.12       -4.98    3.0   11.0ms
 11   -7.251338798679      -10.01       -5.35    1.0   8.12ms
 12   -7.251338798701      -10.66       -5.81    3.0   11.0ms
 13   -7.251338798704      -11.57       -6.21    3.0   11.5ms
 14   -7.251338798705      -12.37       -6.86    1.0   8.45ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.06964736977067967
[ Info: Arnoldi iteration step 2: normres = 0.5677118981861332
[ Info: Arnoldi iteration step 3: normres = 0.7864547034260219
[ Info: Arnoldi iteration step 4: normres = 0.23042000719661962
[ Info: Arnoldi iteration step 5: normres = 0.6173401732206831
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (2.62e-02, 8.92e-02, 5.73e-01, 2.09e-01, 1.41e-02)
[ Info: Arnoldi iteration step 6: normres = 0.22748678238837122
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (4.54e-03, 9.78e-02, 1.44e-01, 1.08e-01, 5.85e-02)
[ Info: Arnoldi iteration step 7: normres = 0.07557935583745279
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.66e-04, 7.11e-03, 9.52e-03, 5.28e-02, 4.63e-02)
[ Info: Arnoldi iteration step 8: normres = 0.07662777893632465
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (5.45e